# 15章　k-最近傍法 
## レシピ15.1　観測値の近傍の発見 


In [ ]:
# ライブラリをロード
from sklearn import datasets
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

# データをロード
iris = datasets.load_iris()
features = iris.data

# 標準化器を作成
standardizer = StandardScaler()

# 特徴量を標準化
features_standardized = standardizer.fit_transform(features)

# 2-最近傍法
nearest_neighbors = NearestNeighbors(n_neighbors=2).fit(features_standardized)

# 観測値を作成
new_observation = [1, 1, 1, 1]

# 最近傍の観測値の距離とインデックスを計算
distances, indices = nearest_neighbors.kneighbors([new_observation])

# 最近傍点を表示
features_standardized[indices]


In [ ]:
# ユークリッド距離を用いて最初の2つの最近傍点を見つける
nearestneighbors_euclidean = NearestNeighbors(
n_neighbors=2, metric='euclidean').fit(features_standardized)

In [ ]:
# 距離を表示
distances

In [ ]:
# ユークリッド距離を用いて、最近傍点を3つ（自身を含む）見つける
nearestneighbors_euclidean = NearestNeighbors(
n_neighbors=3, metric="euclidean").fit(features_standardized)

# それぞれの観測値に対する3つの最近傍点（自身を含む）
# を表すリストのリストを作成
nearest_neighbors_with_self = nearestneighbors_euclidean.kneighbors_graph(
    features_standardized).toarray()

# 自分自身を指している近傍点を削除
for i, x in enumerate(nearest_neighbors_with_self):
    x[i] = 0

# 最初の観測値の2つの最近傍点を表示
nearest_neighbors_with_self[0]

## レシピ15.2　k-最近傍法クラス分類器の作成 


In [ ]:
# ライブラリをロード
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import datasets

# データをロード
iris = datasets.load_iris()
X = iris.data
y = iris.target

# 標準化器を作成
standardizer = StandardScaler()

# 特徴量を標準化
X_std = standardizer.fit_transform(X)

# 近傍値数を5に指定してKNNクラス分類器を訓練
knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1).fit(X_std, y)

# 2つの観測値を作成
new_observations = [[ 0.75, 0.75, 0.75, 0.75],
                    [ 1, 1, 1, 1]]

# 2つの観測値のクラスを予測
knn.predict(new_observations)

In [ ]:
# それぞれの観測値が3つのクラスに属する確率を表示
knn.predict_proba(new_observations)

In [ ]:
knn.predict(new_observations)

## レシピ15.3　最適な近傍サイズの特定 


In [ ]:
# ライブラリをロード
from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV

# データをロード
iris = datasets.load_iris()
features = iris.data
target = iris.target

# 標準化器を作成
standardizer = StandardScaler()

# 最近傍法クラス分類器を作成
knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)

# パイプラインを作成
pipe = Pipeline([("standardizer", standardizer), ("knn", knn)])

# 候補値の空間を作成
search_space = [{"knn__n_neighbors": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}]

# グリッド探索を作成
classifier = GridSearchCV(
    pipe, search_space, cv=5, verbose=0).fit(features_standardized, target)

In [ ]:
# 最良の近傍点数（k）
classifier.best_estimator_.get_params()["knn__n_neighbors"]

## レシピ15.4　半径を用いた最近傍クラス分類器の作成 


In [ ]:
# ライブラリをロード
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import datasets

# データをロード
iris = datasets.load_iris()
features = iris.data
target = iris.target

# 標準化器を作成
standardizer = StandardScaler()

# 特徴量を標準化
features_standardized = standardizer.fit_transform(features)

# 半径を用いる近傍法クラス分類器を訓練
rnn = RadiusNeighborsClassifier(
radius=.5, n_jobs=-1).fit(features_standardized, target)

# 観測値を作成
new_observations = [[1, 1, 1, 1]]

# 観測値のクラスを予測
rnn.predict(new_observations)

## レシピ15.5　近似最近傍の検索 


In [ ]:
# ライブラリをロード
import faiss
import numpy as np
from sklearn import datasets
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

# データをロード
iris = datasets.load_iris()
features = iris.data

# 標準化器を作成
standardizer = StandardScaler()

# 特徴量を標準化
features_standardized = standardizer.fit_transform(features)

# faissのパラメータを設定
n_features = features_standardized.shape[1]
nlist = 3
k = 2

# IVFインデックスを作成
quantizer = faiss.IndexFlatIP(n_features)
index = faiss.IndexIVFFlat(quantizer, n_features, nlist)

# インデックスを訓練し、特徴ベクトルを追加
index.train(features_standardized)
index.add(features_standardized)

# 観測値を作成
new_observation = np.array([[ 1, 1, 1, 1]])

# 2つの最近傍点を検索しインデックスを取得
distances, indices = index.search(new_observation, k)

# 2つの最近傍点の特徴ベクトルを表示
np.array([list(features_standardized[i]) for i in indices[0]])

## レシピ15.6　近似最近傍法の評価 


In [ ]:
# ライブラリをロード
import faiss
import numpy as np
from sklearn import datasets
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

# 近傍点の数
k = 10

# データをロード
iris = datasets.load_iris()
features = iris.data

# 標準化器を作成
standardizer = StandardScaler()

# 特徴量を標準化
features_standardized = standardizer.fit_transform(features)

# KNNで10個の近傍点を作成
nearest_neighbors = NearestNeighbors(n_neighbors=k).fit(features_standardized)

# faissのパラメータを設定
n_features = features_standardized.shape[1]
nlist = 3

# IVFインデックスを作成
quantizer = faiss.IndexFlatIP(n_features)
index = faiss.IndexIVFFlat(quantizer, n_features, nlist)

# インデックスを訓練し、特徴ベクトルを追加
index.train(features_standardized)
index.add(features_standardized)
index.nprobe = 1

# 観測値を作成
new_observation = np.array([[ 1, 1, 1, 1]])

# 正確な最近傍点と観測値の距離とインデックスを取得
knn_distances, knn_indices = nearest_neighbors.kneighbors(new_observation)

# 10個の最近傍点を検索しインデックスを取得
ivf_distances, ivf_indices = index.search(new_observation, k)

# 集合の重複部分を取得
recalled_items = set(list(knn_indices[0])) & set(list(ivf_indices[0]))

# Recall@kを表示
print(f"Recall @k={k}: {len(recalled_items)/k * 100}%")